# EEG Preprocessing Pipeline
This notebook demonstrates how we preprocessed the EEG data obtained from a single subject. To run the code you need to install following packages.

In [ ]:
!pip install mne
!pip install numpy
!pip install pickle

In [ ]:
import mne as mne
import numpy as np
import pickle

### Loading The RAW EEG Data

In [ ]:
# Load EEG data files for eyes-closed and eyes-open conditions
file_path_eyes_closed = r"./data/example_eeg/example_subj_EC_raw.fif.gz"
file_path_eyes_open = r"./data/example_eeg/example_subj_EO_raw.fif.gz"

# Read raw EEG data
raw_eeg_eyes_closed = mne.io.read_raw(file_path_eyes_closed, preload=True, verbose=False)
raw_eeg_eyes_open = mne.io.read_raw(file_path_eyes_open, preload=True, verbose=False)

### Setting The Montage Layout
This function, set_montage_layout, is designed to set the montage layout for raw EEG data using the MNE library in Python. It creates a standard montage layout using make_standard_montage("GSN-HydroCel-129"), which specifies a particular sensor configuration (in this case, GSN HydroCel with 129 electrodes).

In [ ]:
def set_montage_layout(eeg_raw):
    """
    Set the EEG montage layout for the raw data.

    :param eeg_raw: mne.io.Raw - The raw EEG data.
    :return: mne.io.Raw - The EEG data with the montage layout set.
    """
    eeg_raw = eeg_raw.copy()
    montage = mne.channels.make_standard_montage("GSN-HydroCel-129")
    eeg_raw.set_eeg_reference(ref_channels=['Cz'], verbose=False)
    eeg_raw.drop_channels([ch for ch in eeg_raw.ch_names if ch not in montage.ch_names])
    eeg_raw.set_montage(montage)
    return eeg_raw

### Filtering Noise
The filter_powerline_noise function is designed to filter out powerline noise from raw EEG data, a common preprocessing step in EEG analysis. The EEG data is then subjected to a bandpass filter using eeg_raw.filter(1, 70, verbose=False). This filter allows frequencies between 1 Hz and 70 Hz to pass through, effectively removing very low-frequency noise (like slow drifts) and high-frequency noise (like muscle artifacts) outside this range. After bandpass filtering, a notch filter is applied specifically targeting the powerline frequency, which is typically 60 Hz in the USA (or 50 Hz in many other countries).

In [ ]:
def filter_powerline_noise(eeg_raw):
    """
    Filter powerline noise from EEG data.

    :param eeg_raw: mne.io.Raw - The raw EEG data.
    :return: mne.io.Raw - The EEG data with powerline noise filtered out.
    """
    eeg_raw = eeg_raw.copy()
    eeg_raw = eeg_raw.filter(1, 70, verbose=False)
    eeg_raw = eeg_raw.notch_filter(freqs=60, verbose=False)
    return eeg_raw

In [ ]:
# Apply montage layout and filter powerline noise for both conditions
eeg_eyes_closed_montage = set_montage_layout(raw_eeg_eyes_closed)
eeg_eyes_open_montage = set_montage_layout(raw_eeg_eyes_open)

eeg_eyes_closed_filtered = filter_powerline_noise(raw_eeg_eyes_closed)
eeg_eyes_open_filtered = filter_powerline_noise(raw_eeg_eyes_open)

In [ ]:
# Define frequency bands of interest
frequency_bands = {
    'delta': [0.5, 4],
    'theta': [4, 7],
    'alpha': [7, 13],
    'beta': [13, 30],
    'whole_spec': [0.5, 30]
}

### Channel Interpolation
The interpolate_channels function is designed to interpolate channels in EEG data using a specified channel mapping. We have created 3 different channel maps. You can change the mapping to chan_map_R5 (for 5 regions), chan_map_R12 (for 12 regions) and chan_map_R20 (for 20 regions).

In [ ]:
from config.interpolation_maps import chan_map_R5, chan_map_R12, chan_map_R20

def interpolate_channels(eeg_raw, channel_map=chan_map_R12):
    """
    Interpolate channels in EEG data based on a given channel map.

    :param eeg_raw: mne.io.Raw - The raw EEG data.
    :param channel_map: Dict[str, List[str]] - A mapping of channel groups.
    :return: mne.io.Raw - The EEG data with channels interpolated.
    """
    channel_groups = {}
    for key in channel_map.keys():
        channels = [eeg_raw.info['ch_names'].index(x) for x in channel_map[key]]
        channel_groups[key] = channels
    eeg_raw = mne.channels.combine_channels(eeg_raw, groups=channel_groups, method='mean', verbose=False)
    return eeg_raw

In [ ]:
from matplotlib import pyplot as plt
from src.montage_plotter import MontagePlotter

montage_ec = raw_eeg_eyes_closed.get_montage()
ch_pos_ec = montage_ec.get_positions()['ch_pos']
cord_3d = [list(ch_pos_ec[key]) for key in ch_pos_ec.keys()]
ch_names = list(ch_pos_ec.keys())

mp = MontagePlotter(cord_3d, ch_names, chan_map_R5)

mp.plot_regions_2d()
plt.show()

### Frequency Bands of Interest
The filter_freq_bands_of_interest function processes raw EEG data to extract specific frequency bands, each of which may be relevant for different types of analysis.

In [ ]:
def filter_freq_bands_of_interest(eeg_raw, bands_of_interest=frequency_bands, interp_chan=True):
    """
    Filter raw EEG data into different frequency bands of interest.

    :param eeg_raw: mne.io.Raw - The raw EEG data.
    :param bands_of_interest: Dict[str, List[float]] - Frequency bands to filter.
    :param interp_chan: bool - Whether to interpolate channels.
    :return: Dict[str, mne.io.Raw] - A dictionary of filtered EEG data by frequency bands.
    """
    eeg_filtered_bands = dict()
    for band in bands_of_interest.keys():
        band_range = bands_of_interest[band]
        temp_raw = eeg_raw.copy().filter(band_range[0], band_range[1], verbose=False)
        temp_raw = temp_raw.resample(sfreq=250)
        if interp_chan:
            temp_raw = interpolate_channels(temp_raw)
        eeg_filtered_bands[band] = temp_raw
    return eeg_filtered_bands


In [ ]:
# Filter EEG data into frequency bands of interest for both conditions
eeg_eyes_closed_freq_bands = filter_freq_bands_of_interest(eeg_eyes_closed_filtered)
eeg_eyes_open_freq_bands = filter_freq_bands_of_interest(eeg_eyes_open_filtered)

In [ ]:
def save_object_to_pickle(data_object, file_name):
    """
    Save an object to a pickle file.

    :param data_object: Any - The object to be saved.
    :param file_name: str - The file path where the object will be saved.
    """
    try:
        # Opening the file in write-binary mode
        with open(file_name + ".pickle", "wb") as file:
            # Dumping the object to a pickle file
            pickle.dump(data_object, file, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as error:
        # Handling exceptions during the pickle process
        print("Error during pickling object (Possibly unsupported):", error)

# Save the filtered and processed EEG data for eyes-closed condition
save_object_to_pickle(eeg_eyes_closed_freq_bands, r"data/example_eeg/example_subj_EC_preproc")

# Save the filtered and processed EEG data for eyes-open condition
save_object_to_pickle(eeg_eyes_open_freq_bands, r"data/example_eeg/example_subj_EO_preproc")
